# Second Request

In [53]:
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [54]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456",
    database="booktopia"
)
print(mydb)

In [88]:
mycursor = mydb.cursor()

mycursor.execute("""

SELECT
    book_detail.site_id,
    book_detail.English_title,
    book_detail.score,
    book_detail.edition,
    book_detail.stock_status,
    book_detail.ad_publication_year,
    book_detail.solar_publication_year,
    price_history.price,
    rewards.reward
FROM
    book_detail
JOIN
    price_history ON book_detail.site_id = price_history.book_id
JOIN
    rewards ON book_detail.site_id = rewards.site_id
WHERE
    book_detail.stock_status = ' موجود ';
"""
                 )

myresult = mycursor.fetchall()
# for x in myresult:
#   print(x)
import pandas as pd

df = pd.DataFrame(myresult, columns=[i[0] for i in mycursor.description])

In [89]:
df['price'] = pd.to_numeric(df['price'], errors='coerce')
price_threshold = df['price'].quantile(0.2)
cheapest_books = df[df['price'] <= price_threshold]
df = cheapest_books
df.reset_index(drop=True, inplace=True)

In [90]:
df

,site_id,English_title,score,edition,stock_status,ad_publication_year,solar_publication_year,price,reward
0,6580,Collected Poems,3.15,-1,موجود,-1,1394,35000,رودکی مشهور به استاد شاعران، نخستین شاعر مشه...
1,10985,Great House,3.47,1,موجود,2010,1398,45000,برنده ی جایزه ی Anisfield-Wolf سال 2011
2,10985,Great House,3.47,1,موجود,2010,1398,45000,نامزد دریافت جایزه ی اورنج سال 2011
3,10985,Great House,3.47,1,موجود,2010,1398,45000,نازمرد دریافت جایزه ی ملی کتاب آمریکا سال 2010
4,10985,Great House,3.47,1,موجود,2010,1398,45000,نازمرد دریافت جایزه ی بهترین کتاب گودریدز سا...
...,...,...,...,...,...,...,...,...,...
2767,56860,Popo reads a book he knows the value of it,3.65,-1,موجود,2019,1398,35000,تصویرگر: امیر حامد پاژتار
2768,56860,Popo reads a book he knows the value of it,3.44,-1,موجود,2019,1398,35000,تصویرگر: امیر حامد پاژتار
2769,1525,The Festival of Insignificance,3.49,13,موجود,2013,1402,15000,میلان کوندرا، نامزد جایزه نوبل ادبیات و از پ...
2770,1525,The Festival of Insignificance,3.32,2,موجود,2013,1401,15000,میلان کوندرا، نامزد جایزه نوبل ادبیات و از پ...


In [91]:
normalized_reward = df.groupby('site_id').count()['reward']
max_reward = normalized_reward.quantile(0.9)
normalized_reward = normalized_reward / max_reward
normalized_reward[normalized_reward > 1] = 1
normalized_reward[normalized_reward < 0] = 0
normalized_reward

site_id
28        0.25
33        1.00
40        0.25
47        0.75
51        0.25
          ... 
103845    0.25
116447    1.00
117054    0.50
121106    0.25
121148    0.50
Name: reward, Length: 1173, dtype: float64

In [92]:
df = df.groupby('site_id').agg(
    {'score': 'first', 'edition': 'first', 'ad_publication_year': 'first', 'reward': 'count'})
df

,score,edition,ad_publication_year,reward
site_id,,,,
28,3.60,3,2011,1
33,3.50,6,2005,4
40,3.18,1,2006,1
47,3.16,14,2015,3
51,3.61,2,2006,1
...,...,...,...,...
103845,3.94,1,-1,1
116447,3.75,1,-1,4
117054,3.46,2,1989,2


In [95]:
normalized_score = (df['score'] / 5).astype('float64')
normalized_score

site_id
28        0.720
33        0.700
40        0.636
47        0.632
51        0.722
          ...  
103845    0.788
116447    0.750
117054    0.692
121106    0.796
121148    0.624
Name: score, Length: 1173, dtype: float64

In [99]:
normalized_edition = df['edition'] / (2024 - df['ad_publication_year']) * 2
max_edition = normalized_edition.quantile(0.9)
normalized_edition = normalized_edition / max_edition
normalized_edition[normalized_edition > 1] = 1
normalized_edition[normalized_edition < 0] = 0
normalized_edition
# max_edition

site_id
28        0.463287
33        0.633971
40        0.111532
47        1.000000
51        0.223064
            ...   
103845    0.000991
116447    0.000991
117054    0.114719
121106    0.006940
121148    0.001983
Length: 1173, dtype: float64

In [102]:
df['writer_score'] = ((normalized_edition * 0.2) + (normalized_score * 0.6) + normalized_reward * 0.2)
df.sort_values(by='writer_score', ascending=False)

,score,edition,ad_publication_year,reward,writer_score
site_id,,,,,
706,4.04,12,1998,4,0.870115
616,3.91,24,2003,18,0.869200
1606,3.89,85,1958,4,0.866800
23248,3.96,4,2015,6,0.853651
846,3.76,39,1998,28,0.851200
...,...,...,...,...,...
21952,0.00,1,2011,1,0.080886
13103,0.00,20,-1,1,0.053966
13149,0.00,1,1840,1,0.052182
